<a href="https://colab.research.google.com/github/ohilikeit/Dacon_code_similarity/blob/main/huggingface_CodeBERTa_small_v1__TPU%2B%EC%83%88%EB%A1%9C%EC%9A%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/code_similarity

/content/drive/MyDrive/code_similarity


In [ ]:
! pip install transformers datasets
! pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl
import os
import random
assert os.environ['COLAB_TPU_ADDR']
import transformers
import torch
import torch_xla
import torch_xla.core.xla_model as xm
from transformers.utils.dummy_pt_objects import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_scheduler
import pandas as pd
import datasets
import numpy as np
from datasets import load_dataset, load_metric
from transformers.utils import logging

dev = xm.xla_device()
seed = 42
random.seed(seed)
np.random.seed(seed)
transformers.set_seed(seed)
transformers.enable_full_determinism(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
MODEL = "huggingface/CodeBERTa-small-v1"
MAX_LEN = 512

tokenizer = RobertaTokenizer.from_pretrained(MODEL)
dataset = load_dataset('csv', data_files={'train': 'train_data', 'test': 'val_data'}) # 읽어오려면 .csv 형태면 안된다. 저장할때 빼고 저장할 것 바이너리 파일. 

logging.set_verbosity_error() # 코드 실행시 script가 길어져서 생기는 오류를 방지하기 위한 코드 

def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

##### 둘 중 하나만 #####
#dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
#dataset.save_to_disk('dataset') # 일단 저장.. 
dataset = datasets.load_from_disk("dataset")

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def compute_metrics(p):
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1),)
    return output

logging.set_verbosity_info() # 진행바 확인을 위해 logging 재설정 

#MODEL = '/content/drive/MyDrive/code_similarity/experiments_eval_loss/checkpoint-11000' # 34000 checkpoint 이후 오류가 생겨 다시 불러와서 재학습시켰다. 
model = RobertaForSequenceClassification.from_pretrained(MODEL).to(dev)
args = TrainingArguments(
    'experiments_eval_loss',
    evaluation_strategy = 'steps',
    save_steps = 2000,
    eval_steps = 2000,
    load_best_model_at_end = True,
    metric_for_best_model='eval_loss', # loss or eval_loss로 해보자, 할꺼면 greater_is_better = False로, 낮아져야하는거니깐 + step을 5정도로 늘리자 
    greater_is_better = False,
    per_device_train_batch_size=32, # 가능한 최대 크기 배치 사이즈임 
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    warmup_steps=1000,
    lr_scheduler_type="cosine",
    learning_rate = 5e-5,
    weight_decay= 0.1,
    do_train=True,
    do_eval=True,
    save_strategy='steps',
    logging_strategy="steps",
    tpu_num_cores=8,                 # 8코어를 사용하고 있는지는 잘 모르겠다. 
    #gradient_accumulation_steps = 4, # gpu 메모리 줄이는법, 배치 사이즈랑 상충관계. 1 * 64 = 4 * 16 
    #gradient_checkpointing=True, # forward 시에 활성값들 까먹고 backward에 재계산, 메모리 효율적, 시간은 더 걸림. 
    seed=42
)

trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)])

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3365893
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 210370


Step,Training Loss,Validation Loss,Accuracy
2000,0.085100,0.057841,0.982237
4000,0.071300,0.053439,0.983888
6000,0.067900,0.049420,0.981565
8000,0.057200,0.046761,0.985162
10000,0.052400,0.045875,0.985049
12000,0.057400,0.067207,0.978438
14000,0.054700,0.066098,0.982084
16000,0.055600,0.052818,0.980567


***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-2000
Configuration saved in experiments_eval_loss/checkpoint-2000/config.json
Model weights saved in experiments_eval_loss/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-2000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-2000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-4000
Configuration saved in experiments_eval_loss/checkpoint-4000/config.json
Model weights saved in experiments_eval_loss/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-4000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-4000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
 

TrainOutput(global_step=16000, training_loss=0.06851361060142518, metrics={'train_runtime': 21795.5042, 'train_samples_per_second': 308.861, 'train_steps_per_second': 9.652, 'total_flos': 6.431391555297792e+16, 'train_loss': 0.06851361060142518, 'epoch': 0.15})

In [ ]:
trainer.train() # 밑에 꺼 max_len만 512로 바꿨다. 

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3365893
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 210370


Step,Training Loss,Validation Loss,Accuracy
500,0.170200,0.068907,0.977405
1000,0.095300,0.097291,0.965171
1500,0.089000,0.072425,0.979431
2000,0.080000,0.055360,0.983117
2500,0.076000,0.051827,0.983942
3000,0.070200,0.060119,0.982351
3500,0.063600,0.059898,0.984145
4000,0.067800,0.063607,0.979480


***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-500
Configuration saved in experiments_eval_loss/checkpoint-500/config.json
Model weights saved in experiments_eval_loss/checkpoint-500/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-500/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-1000
Configuration saved in experiments_eval_loss/checkpoint-1000/config.json
Model weights saved in experiments_eval_loss/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-1000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batc

TrainOutput(global_step=4000, training_loss=0.0890111541748047, metrics={'train_runtime': 15234.8818, 'train_samples_per_second': 441.867, 'train_steps_per_second': 13.808, 'total_flos': 1.6098696724964352e+16, 'train_loss': 0.0890111541748047, 'epoch': 0.04})

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

88

In [ ]:
trainer.train() # 0.65 나옴 2시간 

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3365893
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 210370


Step,Training Loss,Validation Loss,Accuracy
2000,0.096500,0.073349,0.976229
4000,0.074100,0.059417,0.980236
6000,0.068100,0.057774,0.980874
8000,0.057400,0.052624,0.982697
10000,0.055400,0.048165,0.981906
12000,0.055400,0.073156,0.977198
14000,0.054800,0.054029,0.983176
16000,0.052900,0.053371,0.984411
18000,0.055700,0.059199,0.981723


***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-2000
Configuration saved in experiments_eval_loss/checkpoint-2000/config.json
Model weights saved in experiments_eval_loss/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-2000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-2000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 32
Saving model checkpoint to experiments_eval_loss/checkpoint-4000
Configuration saved in experiments_eval_loss/checkpoint-4000/config.json
Model weights saved in experiments_eval_loss/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-4000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-4000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
 

TrainOutput(global_step=18000, training_loss=0.07074858305189345, metrics={'train_runtime': 9972.6851, 'train_samples_per_second': 675.022, 'train_steps_per_second': 21.095, 'total_flos': 3.8150610812928e+16, 'train_loss': 0.07074858305189345, 'epoch': 0.17})

In [ ]:
trainer.train() # 이건 loss 기준으로 새로 해본거 5시간 걸림. 

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3365893
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 52594


Step,Training Loss,Validation Loss,Accuracy
500,0.178000,0.067939,0.976506
1000,0.068700,0.065535,0.976219
1500,0.054400,0.058761,0.980098
2000,0.048000,0.046534,0.981224
2500,0.043900,0.041628,0.984584
3000,0.039700,0.042472,0.984634
3500,0.036900,0.041390,0.985286
4000,0.037100,0.035270,0.986857
4500,0.035000,0.033718,0.987751
5000,0.035500,0.034718,0.987238


***** Running Evaluation *****
  Num examples = 202391
  Batch size = 128
Saving model checkpoint to experiments_eval_loss/checkpoint-500
Configuration saved in experiments_eval_loss/checkpoint-500/config.json
Model weights saved in experiments_eval_loss/checkpoint-500/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-500/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 128
Saving model checkpoint to experiments_eval_loss/checkpoint-1000
Configuration saved in experiments_eval_loss/checkpoint-1000/config.json
Model weights saved in experiments_eval_loss/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in experiments_eval_loss/checkpoint-1000/tokenizer_config.json
Special tokens file saved in experiments_eval_loss/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Ba

KeyboardInterrupt: ignored

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3365893
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 52594


Step,Training Loss,Validation Loss,Accuracy
2000,0.051500,0.039323,0.985360
4000,0.036900,0.034815,0.986907
6000,0.031900,0.030895,0.988404
8000,0.028400,0.032210,0.987578
10000,0.026600,0.034243,0.988922
12000,0.026700,0.032762,0.988838
14000,0.023700,0.030774,0.988754
16000,0.024100,0.025333,0.990133
18000,0.021900,0.028599,0.989392
20000,0.021100,0.027832,0.989347


***** Running Evaluation *****
  Num examples = 202391
  Batch size = 128
Saving model checkpoint to experiments/checkpoint-2000
Configuration saved in experiments/checkpoint-2000/config.json
Model weights saved in experiments/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in experiments/checkpoint-2000/tokenizer_config.json
Special tokens file saved in experiments/checkpoint-2000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 128
Saving model checkpoint to experiments/checkpoint-4000
Configuration saved in experiments/checkpoint-4000/config.json
Model weights saved in experiments/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in experiments/checkpoint-4000/tokenizer_config.json
Special tokens file saved in experiments/checkpoint-4000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 202391
  Batch size = 128
Saving model checkpoint to experiments/checkpoint-6000
Configuration saved in ex

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

88

In [ ]:
import pandas as pd
import datasets

logging.set_verbosity_error() 
TEST = "sample_test"
SUB = "sample_submission.csv"

#test_dataset = load_dataset("csv", data_files=TEST)['train']
#test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

#test_dataset.save_to_disk('test_dataset_v2')
test_dataset = datasets.load_from_disk("test_dataset")

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('prediction_tpu.csv', index=False)
df

,pair_id,similar
0,1,1
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
179695,179696,1
179696,179697,1
179697,179698,1
179698,179699,1
